In [1]:
def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

def make_derivatives(f,vl,kmax):
    # f is the expression (of variables in vl) for which we are taking derivatives
    # vl is the list of variables we use for partial derivatives
    # kl is the max number of mutations allowed for each branch type.
    
    
    dic = {}
    # dictionary to store values and probably to return
    
    maxOrder=len(vl)*kmax
    
    def add_one_each(l):
        dim = len(l)
        addUnit = [[1 if x==y else 0 for x in range(dim)] for y in range(dim)]
        newVals = [map(lambda x:sum(x), zip(l,x)) for x in addUnit]
        newVals = [ (l,x)  for x in newVals if max(x)<=kmax]
        return newVals

    def add_symbolic(sl,el):
        if tuple(el) in dic.keys():
             return None
        else:
            dl = map(lambda (x,y): y - x,zip(sl,el))
            
            g = dic[tuple(sl)];
            for (a,b) in zip(vl,dl):
                g = diff(g,a,b)
            dic[tuple(el)] = g
            return el
                

    
    level0 = [0,0,0,0]
    order = sum(level0)    
    while (order < maxOrder+1):
        
        #we have to calculate the derivatives on each level.when we level up and start the new loop, we have to 
        #play the game... taken to the next level
        if (order == 0):
            thisLevel = [level0]
            dic[tuple(level0)] = f
        else:
            thisLevel = nextLevel
                
        nextLevel = [ add_one_each(stage) for stage in thisLevel]
        nextLevel = flatten(nextLevel)
        nextLevel = [add_symbolic(a,b) for (a,b) in nextLevel]
        nextLevel = [ stage for stage in nextLevel if stage is not None ]
            
        #nextLevel= list( set( [tuple(stage) for stage in nextLevel] ) )
        # this duplication will be taken care of by checking that the next stage higher is not already
        # in the dictionary of values.
        
        

        order = order + 1
        
    return dic   

In [2]:
def get_sym_prob(pathDic,key,mutVarl):
    thetaProd = reduce(lambda x, y: x * y, [(b/2)^(a) for (a,b) in zip(key,mutVarl)], 1)
    factSum = sum(key);
    return pathDic[key]*thetaProd/factorial(factSum)*(-1)^(factSum)

def get_num_prob(pathDic,key,mutVarl,subDic):
    symProb = get_sym_prob(pathDic,key,mutVarl);
    numProb = symProb.substitute(subDic)
    return numProb



In [3]:

pathEquationsE = ['C_ancestor^2*C_derived*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_fix)*(E + M + 2*theta_fix))', '2*C_ancestor^2*C_derived*M/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_B))', 'C_ancestor^2*C_derived*M/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^2*C_derived*E/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_B))', 'C_ancestor^2*C_derived*E/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_B + theta_fix)*(C_ancestor + 2*theta_fix))', 'C_ancestor^2*C_derived*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*M + 2*theta_A + theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*M + 2*theta_A + theta_fix)*(E + M + 2*theta_fix))', '2*C_ancestor^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + 2*M + 2*theta_A + theta_fix)*(E + M + 2*theta_A))', '2*C_ancestor^3*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + 2*M + 2*theta_A + theta_fix)*(E + M + 2*theta_A))', '2*C_ancestor^3*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', '4*C_ancestor^3*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', '2*C_ancestor^3*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', '4*C_ancestor^3*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', 'C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', '2*C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(C_derived + E + 2*M + 2*theta_A + theta_fix))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_A)*(E + M + 2*theta_A))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_A)*(E + M + 2*theta_A))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_B))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_AB))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_A))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_B))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_AB))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + theta_A + theta_AB + theta_B)*(C_ancestor + 2*theta_A))', '2*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A))', '2*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A))', '2*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', 'C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB))', '4*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', 'C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^3*M^2/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', '2*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', 'C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB))', '4*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', 'C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^3*E*M/((3*C_ancestor + E + M + 2*theta_A + 2*theta_B)*(3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', 'C_ancestor^3*E/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '1/2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', '1/2*C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', 'C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))']
pathEquationsE= [str(x) for x in pathEquationsE]

pathEquationsR = ['C_ancestor*C_derived^2*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(E + M + 2*theta_fix))', '2*C_ancestor*C_derived^2*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_B))', '2*C_ancestor^2*C_derived*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_B))', 'C_ancestor*C_derived^2*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix))', 'C_ancestor^2*C_derived*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix))', '2*C_ancestor^3*E*M/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix))', 'C_ancestor^3*E*M/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix))', '2*C_ancestor^3*E/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(C_derived + E + M + 2*theta_B + theta_fix))', 'C_ancestor^3*E/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(C_derived + E + M + 2*theta_B + theta_fix))', 'C_ancestor*C_derived^2*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(C_derived + E + 2*theta_fix)*(E + M + 2*theta_fix))', 'C_ancestor^2*C_derived*E/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(E + M + 2*theta_fix))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor*C_derived^2*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + M + 2*theta_A + theta_fix)*(E + M + 2*theta_A))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '2*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix))', '2*C_ancestor^2*C_derived*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix))', '4*C_ancestor^2*C_derived*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A)*(C_derived + E + M + 2*theta_A + theta_fix))', 'C_ancestor^2*C_derived*E/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^2*C_derived*E/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + E + 2*M + 2*theta_A + theta_fix)*(C_ancestor + 2*theta_A))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(E + M + 2*theta_A))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_B))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_B))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_AB))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_AB))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '4*C_ancestor^2*C_derived*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '4*C_ancestor^2*C_derived*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '4*C_ancestor^2*C_derived*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + theta_A + theta_AB + theta_B))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A)*(E + M + 2*theta_A))', '2*C_ancestor*C_derived^2*E*M/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_A + theta_fix)*(E + M + 2*theta_A))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '2*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + M + 2*theta_A + theta_fix))', '2*C_ancestor^2*C_derived*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_A + theta_fix))', '4*C_ancestor^2*C_derived*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(C_derived + E + M + 2*theta_A + theta_fix))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_B))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_B))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix)*(C_derived + E + 2*theta_fix))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix))', '2*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_B + theta_fix))', '8*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_B))', '8*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_B))', '8*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_AB))', '8*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_AB))', '8*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A))', '8*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B)*(C_derived + E + 2*theta_A))', '8*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B))', '8*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B))', '8*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + theta_A + theta_AB + theta_B))', '2*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '2*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_fix))', '4*C_derived^3*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '4*C_ancestor*C_derived^2*E*M^2/((C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix)*(C_derived + E + 2*theta_A))', '2*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix))', '4*C_ancestor^2*C_derived*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B)*(3*C_derived + E + 2*theta_A + theta_fix))', '2*C_ancestor^3*E*M^2/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', 'C_ancestor^3*E*M^2/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M^2/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', 'C_ancestor^3*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '2*C_ancestor^3*E*M^2/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(6*C_derived + E + 2*theta_A + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '2*C_ancestor^3*E*M/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', 'C_ancestor^3*E*M/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '4*C_ancestor^3*E*M/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', 'C_ancestor^3*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', '2*C_ancestor^3*E*M/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A)*(3*C_derived + E + M + 2*theta_A + 2*theta_B))', 'C_ancestor^3*E/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '1/2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_B)*(3*C_ancestor + 2*theta_B + theta_fix)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_B))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_AB))', '2*C_ancestor^3*E/((3*C_ancestor + theta_A + theta_AB + theta_B)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))', '1/2*C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_fix))', 'C_ancestor^3*E/((3*C_ancestor + 2*theta_A + theta_fix)*(3*C_ancestor + theta_A + theta_B)*(C_ancestor + C_derived + E + 2*M + 2*theta_A + 2*theta_B)*(C_ancestor + 2*theta_A))']
pathEquationsR = [str(a) for a in pathEquationsR]
print (len(pathEquationsE), len(pathEquationsR))

(51, 95)


In [4]:
var('C_ancestor, C_derived, E, M, R, theta_A, theta_B, theta_AB, theta_fix, T');
pathEquationsE = [ SR(a) for a in pathEquationsE];
pathEquationsR=  [ SR(a) for a in pathEquationsR];
print pathEquationsE[0].has(R)
print pathEquationsR[0].has(R)


False
False


In [5]:
invertedPathsE= [inverse_laplace(a/E,E,T) for a in pathEquationsE];
invertedPathsR = [inverse_laplace(a/E,E,T) for a in pathEquationsR];

#invertedGiac = [inverse_laplace(a/E,E,T,algorithm='sympy') for a in pathEquations];
# invertedGiac = [gia c('invlaplace(%s/E,E,T)'%a) for a in pathEquations];
# print invertedPaths

In [10]:
print(len(invertedPathsR))

95


In [24]:
mutList = [theta_A,theta_B,theta_AB,theta_fix]; 
kmax = 1

#for numeric evaluations, currently all pathwise mutation configuration probabilities
# are stored in the dictionary symbolically.
substitutionDic ={C_ancestor:1, C_derived:1, M:234/100/2, theta_A:12/10/2,
                  theta_B:12/10/2, theta_AB:12/10/2, theta_fix:12/10/2, T:14/10}

testE = make_derivatives(invertedPathsE[0],mutList,kmax); #print test;
testR = make_derivatives(invertedPathsR[0],mutList,kmax); #print test;
print len(testE)
print len(testR)


16
16


In [7]:
pathDerDicE = [ make_derivatives(a,mutList,kmax) for a in invertedPathsE];
pathDerDicR = [ make_derivatives(a,mutList,kmax) for a in invertedPathsR];

In [8]:
print len(pathDerDicR)

95


In [25]:
testSymE = get_sym_prob(pathDerDicE[11],(1,1,0,0),mutList);
testNumE = get_num_prob(pathDerDicE[11],(1,1,0,0),mutList,substitutionDic)


testSymR = get_sym_prob(pathDerDicR[11],(1,1,0,0),mutList);
testNumR = get_num_prob(pathDerDicR[11],(1,1,0,0),mutList,substitutionDic)


print n(testSymR.substitute(substitutionDic))
print n(testNumR)

0.0000416742786675746
0.0000416742786675746


In [26]:

a = sum([get_num_prob(a,(0,0,0,0),mutList,substitutionDic) for a in pathDerDicE]);
b = sum([get_num_prob(b,(0,0,0,0),mutList,substitutionDic) for b in pathDerDicR]);
print n(a);
print n(b);

0.0764073865047889
0.0764073865047889


In [23]:
sum([n(i.substitute({C_ancestor:1, C_derived:1, M:234/100/2, theta_A:12/10/2,
                  theta_B:12/10/2, theta_AB:12/10/2, theta_fix:12/10/2, T:14/10})) for i in invertedPathsE])

0.0764073865047889

In [18]:
12/10/2

3/5

In [49]:
def get_total_prob(pathDerDic,key,substitutionDic):
    pathProbs = [ get_num_prob(a,key,mutList,substitutionDic) for a in pathDerDic]
    return sum(pathProbs)
    
keysAllE = pathDerDicE[0].keys();
keysAllR = pathDerDicR[0].keys()


totalProbsE = [(key, n(get_total_prob(pathDerDicE,key,substitutionDic))) for key in keysAllE];
totalProbsR = [(key, n(get_total_prob(pathDerDicR,key,substitutionDic))) for key in keysAllR];

print totalProbsE
print totalProbsR

[((1, 1, 0, 1), 0.000117498755071994), ((1, 0, 1, 1), 0.000000000000000), ((1, 0, 0, 0), 0.0407582211332397), ((0, 1, 1, 0), 0.00126099325477335), ((1, 1, 0, 0), 0.00525562279475192), ((0, 0, 0, 1), 0.0148333754950207), ((0, 1, 0, 1), 0.00163840935759379), ((1, 0, 0, 1), 0.00208101329764291), ((0, 0, 1, 0), 0.00738377566695452), ((1, 1, 1, 1), 0.000000000000000), ((0, 0, 0, 0), 0.107907095247685), ((0, 1, 0, 0), 0.0306661923169408), ((1, 0, 1, 0), 0.00136155430724118), ((1, 1, 1, 0), 0.000179326956839008), ((0, 0, 1, 1), 0.000000000000000), ((0, 1, 1, 1), 0.000000000000000)]
[((1, 1, 0, 1), 0.000117498755071994), ((1, 0, 1, 1), 0.000000000000000), ((1, 0, 0, 0), 0.0407582211332397), ((0, 1, 1, 0), 0.00126099325477335), ((1, 1, 0, 0), 0.00525562279475192), ((0, 0, 0, 1), 0.0148333754950207), ((0, 1, 0, 1), 0.00163840935759379), ((1, 0, 0, 1), 0.00208101329764291), ((0, 0, 1, 0), 0.00738377566695452), ((1, 1, 1, 1), 0.000000000000000), ((0, 0, 0, 0), 0.107907095247685), ((0, 1, 0, 0), 0.

In [50]:
print sum ([ b for (a,b) in totalProbsE])
print sum ([ b for (a,b) in totalProbsR])

0.213443078583755
0.213443078583755
